In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np 
import pandas as pd

from pathlib import Path

In [3]:
train_dir = Path('E:\data\RSNA2024')

In [4]:
class CFG:
    random_seed = 42

    image_size = 256
    
    ROOT_FOLDER = train_dir / 'original'
    DEST_FOLDER = train_dir
    PNG_DIR = ROOT_FOLDER / f'pngs_{image_size}'
    IMAGES_DIR = ROOT_FOLDER / 'train_images'
    TRAIN_CSV = ROOT_FOLDER / 'train.csv'
    FILES_CSV = ROOT_FOLDER / 'train_files.csv'
    TRAIN_DESC_CSV = ROOT_FOLDER / 'train_series_descriptions.csv'
    COORDS_CSV = ROOT_FOLDER / 'train_label_coordinates.csv'

### train_df

In [5]:
train_df = pd.read_csv(CFG.TRAIN_CSV)
train_desc_df = pd.read_csv(CFG.TRAIN_DESC_CSV)

train_df.shape, train_desc_df.shape

((1975, 26), (6294, 3))

In [6]:
train_df.head()

,study_id,spinal_canal_stenosis_l1_l2,spinal_canal_stenosis_l2_l3,spinal_canal_stenosis_l3_l4,spinal_canal_stenosis_l4_l5,spinal_canal_stenosis_l5_s1,left_neural_foraminal_narrowing_l1_l2,left_neural_foraminal_narrowing_l2_l3,left_neural_foraminal_narrowing_l3_l4,left_neural_foraminal_narrowing_l4_l5,...,left_subarticular_stenosis_l1_l2,left_subarticular_stenosis_l2_l3,left_subarticular_stenosis_l3_l4,left_subarticular_stenosis_l4_l5,left_subarticular_stenosis_l5_s1,right_subarticular_stenosis_l1_l2,right_subarticular_stenosis_l2_l3,right_subarticular_stenosis_l3_l4,right_subarticular_stenosis_l4_l5,right_subarticular_stenosis_l5_s1
0,4003253,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
1,4646740,Normal/Mild,Normal/Mild,Moderate,Severe,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Severe,Normal/Mild,Normal/Mild,Moderate,Moderate,Moderate,Normal/Mild
2,7143189,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
3,8785691,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild
4,10728036,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,...,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Normal/Mild,Moderate,Normal/Mild


In [7]:
cols = train_df.columns[1:]
# first = [c.split('_')[:-2] for c in cols]
# last = [c.split('_')[-2:] for c in cols]

cols = [c.split('_') for c in cols]
cols = [''.join([i[0] if len(i) > 2 else i for i in c]).upper() for c in cols]

cols = ['study_id'] + cols

cols[:5]

['study_id', 'SCSL1L2', 'SCSL2L3', 'SCSL3L4', 'SCSL4L5']

In [8]:
dict(zip(train_df.columns, cols))

{'study_id': 'study_id',
 'spinal_canal_stenosis_l1_l2': 'SCSL1L2',
 'spinal_canal_stenosis_l2_l3': 'SCSL2L3',
 'spinal_canal_stenosis_l3_l4': 'SCSL3L4',
 'spinal_canal_stenosis_l4_l5': 'SCSL4L5',
 'spinal_canal_stenosis_l5_s1': 'SCSL5S1',
 'left_neural_foraminal_narrowing_l1_l2': 'LNFNL1L2',
 'left_neural_foraminal_narrowing_l2_l3': 'LNFNL2L3',
 'left_neural_foraminal_narrowing_l3_l4': 'LNFNL3L4',
 'left_neural_foraminal_narrowing_l4_l5': 'LNFNL4L5',
 'left_neural_foraminal_narrowing_l5_s1': 'LNFNL5S1',
 'right_neural_foraminal_narrowing_l1_l2': 'RNFNL1L2',
 'right_neural_foraminal_narrowing_l2_l3': 'RNFNL2L3',
 'right_neural_foraminal_narrowing_l3_l4': 'RNFNL3L4',
 'right_neural_foraminal_narrowing_l4_l5': 'RNFNL4L5',
 'right_neural_foraminal_narrowing_l5_s1': 'RNFNL5S1',
 'left_subarticular_stenosis_l1_l2': 'LSSL1L2',
 'left_subarticular_stenosis_l2_l3': 'LSSL2L3',
 'left_subarticular_stenosis_l3_l4': 'LSSL3L4',
 'left_subarticular_stenosis_l4_l5': 'LSSL4L5',
 'left_subarticular_ste

In [9]:
train_df.rename(columns=dict(zip(train_df.columns, cols)), inplace=True)
train_df.shape

(1975, 26)

In [10]:
train_df.iloc[0]

study_id        4003253
SCSL1L2     Normal/Mild
SCSL2L3     Normal/Mild
SCSL3L4     Normal/Mild
SCSL4L5     Normal/Mild
SCSL5S1     Normal/Mild
LNFNL1L2    Normal/Mild
LNFNL2L3    Normal/Mild
LNFNL3L4    Normal/Mild
LNFNL4L5       Moderate
LNFNL5S1    Normal/Mild
RNFNL1L2    Normal/Mild
RNFNL2L3    Normal/Mild
RNFNL3L4       Moderate
RNFNL4L5       Moderate
RNFNL5S1    Normal/Mild
LSSL1L2     Normal/Mild
LSSL2L3     Normal/Mild
LSSL3L4     Normal/Mild
LSSL4L5        Moderate
LSSL5S1     Normal/Mild
RSSL1L2     Normal/Mild
RSSL2L3     Normal/Mild
RSSL3L4     Normal/Mild
RSSL4L5     Normal/Mild
RSSL5S1     Normal/Mild
Name: 0, dtype: object

In [11]:
train_df.study_id.nunique()

1975

In [12]:
vals = {'Normal/Mild': 'N', 'Moderate': 'M', 'Severe': 'S'}
vals

{'Normal/Mild': 'N', 'Moderate': 'M', 'Severe': 'S'}

In [13]:
train_df[cols[1:]] = train_df[cols[1:]].replace(vals)

In [14]:
train_df.sample(2)

,study_id,SCSL1L2,SCSL2L3,SCSL3L4,SCSL4L5,SCSL5S1,LNFNL1L2,LNFNL2L3,LNFNL3L4,LNFNL4L5,...,LSSL1L2,LSSL2L3,LSSL3L4,LSSL4L5,LSSL5S1,RSSL1L2,RSSL2L3,RSSL3L4,RSSL4L5,RSSL5S1
1271,2780118855,N,N,N,N,N,N,N,N,N,...,N,N,N,N,N,N,N,N,M,N
155,330993169,N,N,N,N,N,N,N,N,N,...,N,N,N,M,N,N,N,N,M,N


### coordinates_df

In [15]:
coords_df = pd.read_csv(CFG.COORDS_CSV)
files_df = pd.read_csv(CFG.FILES_CSV)

coords_df.shape, files_df.shape

((48692, 7), (147218, 10))

In [16]:
coords_df.rename(columns={'instance_number': 'instance'}, inplace=True)

In [17]:
coords_df.study_id.nunique(), coords_df.condition.nunique(), coords_df.level.nunique()

(1974, 5, 5)

In [18]:
coords_df.condition.unique(), coords_df.level.unique()

(array(['Spinal Canal Stenosis', 'Right Neural Foraminal Narrowing',
        'Left Neural Foraminal Narrowing', 'Left Subarticular Stenosis',
        'Right Subarticular Stenosis'], dtype=object),
 array(['L1/L2', 'L2/L3', 'L3/L4', 'L4/L5', 'L5/S1'], dtype=object))

In [19]:
coords_df.series_id.nunique()

6291

In [20]:
coords_df['ss_id'] = coords_df.apply(lambda row: f'{str(row["study_id"])}_{str(row["series_id"])}', axis=1)
coords_df['instance_id'] = coords_df.apply(lambda row: f'{str(row["study_id"])}_{str(row["series_id"])}_{str(row["instance"])}', axis=1)

train_desc_df['ss_id'] = train_desc_df.apply(lambda row: f'{str(row["study_id"])}_{str(row["series_id"])}', axis=1)

In [21]:
train_desc_df.sample()

,study_id,series_id,series_description,ss_id
4545,3110826149,3762854585,Sagittal T1,3110826149_3762854585


In [22]:
coords_df.sample(2)

,study_id,series_id,instance,condition,level,x,y,ss_id,instance_id
31505,2793936300,1334281856,2,Right Subarticular Stenosis,L1/L2,271.818182,348.636364,2793936300_1334281856,2793936300_1334281856_2
12520,1117361192,2041546893,13,Right Subarticular Stenosis,L3/L4,143.577763,172.660300,1117361192_2041546893,1117361192_2041546893_13


In [23]:
# rename condition
coords_df['condition'] = coords_df.apply(lambda row: ''.join([w[0] for w in row['condition'].split(' ')]), axis=1)

In [24]:
# rename level
coords_df['level'] = coords_df.level.apply(lambda l: ''.join(l.split('/')))

In [25]:
coords_df['cl'] = coords_df['condition'] + coords_df['level']

In [26]:
coords_df.condition.nunique()

5

In [27]:
coords_df.head(10)

,study_id,series_id,instance,condition,level,x,y,ss_id,instance_id,cl
0,4003253,702807833,8,SCS,L1L2,322.831858,227.964602,4003253_702807833,4003253_702807833_8,SCSL1L2
1,4003253,702807833,8,SCS,L2L3,320.571429,295.714286,4003253_702807833,4003253_702807833_8,SCSL2L3
2,4003253,702807833,8,SCS,L3L4,323.030303,371.818182,4003253_702807833,4003253_702807833_8,SCSL3L4
3,4003253,702807833,8,SCS,L4L5,335.292035,427.327434,4003253_702807833,4003253_702807833_8,SCSL4L5
4,4003253,702807833,8,SCS,L5S1,353.415929,483.964602,4003253_702807833,4003253_702807833_8,SCSL5S1
5,4003253,1054713880,4,RNFN,L4L5,187.961759,251.839388,4003253_1054713880,4003253_1054713880_4,RNFNL4L5
6,4003253,1054713880,4,RNFN,L5S1,198.240918,285.613767,4003253_1054713880,4003253_1054713880_4,RNFNL5S1
7,4003253,1054713880,5,RNFN,L3L4,187.227533,210.722753,4003253_1054713880,4003253_1054713880_5,RNFNL3L4
8,4003253,1054713880,6,RNFN,L1L2,194.569790,127.755258,4003253_1054713880,4003253_1054713880_6,RNFNL1L2
9,4003253,1054713880,6,RNFN,L2L3,191.632887,165.934990,4003253_1054713880,4003253_1054713880_6,RNFNL2L3


In [28]:
train_desc_df.series_description.unique()

array(['Sagittal T2/STIR', 'Sagittal T1', 'Axial T2'], dtype=object)

In [29]:
# train_desc_df[train_desc_df['ss_id'] == '4003253702807833'].series_description.values[0]

In [30]:
coords_df['plane'] = coords_df.apply(lambda row: train_desc_df[train_desc_df['ss_id'] == row['ss_id']].series_description.values[0], axis=1)

In [31]:
coords_df.sample(5)

,study_id,series_id,instance,condition,level,x,y,ss_id,instance_id,cl,plane
45044,3966348292,1948287748,3,LNFN,L1L2,336.083736,148.405797,3966348292_1948287748,3966348292_1948287748_3,LNFNL1L2,Sagittal T1
5107,436916913,3871363998,11,LNFN,L5S1,224.954338,272.365297,436916913_3871363998,436916913_3871363998_11,LNFNL5S1,Sagittal T1
14050,1261271580,1688147738,16,LSS,L2L3,131.863971,137.411765,1261271580_1688147738,1261271580_1688147738_16,LSSL2L3,Axial T2
22018,1979480595,192414264,20,RSS,L1L2,285.451327,358.796460,1979480595_192414264,1979480595_192414264_20,RSSL1L2,Axial T2
43926,3873246148,1849086411,45,LSS,L5S1,364.363484,321.608040,3873246148_1849086411,3873246148_1849086411_45,LSSL5S1,Axial T2


In [32]:
# check canal stenosis is noy only in axial plane
coords_df[(coords_df.condition == 'SCS') & (coords_df.plane != 'Axial T2')].sample()

,study_id,series_id,instance,condition,level,x,y,ss_id,instance_id,cl,plane
31474,2782244888,3788734728,8,SCS,L1L2,181.007625,90.283224,2782244888_3788734728,2782244888_3788734728_8,SCSL1L2,Sagittal T2/STIR


In [33]:
# get the positive slices
coords_df.groupby(['study_id','series_id']).instance.unique()

study_id    series_id 
4003253     702807833                              [8]
            1054713880               [4, 5, 6, 11, 12]
            2448190387          [3, 4, 11, 19, 28, 35]
4646740     3201256954    [15, 16, 22, 28, 29, 34, 40]
            3486248476           [5, 6, 7, 15, 16, 17]
                                      ...             
4287160193  1507070277                             [8]
            1820446240          [4, 9, 10, 16, 22, 28]
4290709089  3274612423                             [9]
            3390218084    [2, 3, 5, 6, 10, 15, 20, 21]
            4237840455                  [4, 5, 11, 12]
Name: instance, Length: 6291, dtype: object

In [34]:
coords_df.ss_id.nunique(), coords_df.instance_id.nunique()

(6291, 24546)

### files_df

In [35]:
files_df = pd.read_csv(CFG.FILES_CSV)

In [36]:
files_df.sample(5)

,study_id,series_id,image,instancenumber,rows,columns,slicethickness,spacingbetweenslices,patientposition,seriesdescription
28625,176605889,562357601,15,15,320,320,4.0,4.40,HFS,T1
144085,933831851,394729551,16,16,576,576,4.0,4.40,HFS,NaN
23443,1644309505,1627302771,16,16,384,384,4.0,4.40,HFS,NaN
118560,4105233324,1005903160,9,9,384,384,4.0,4.40,HFS,T2
62140,26342422,2528347280,9,9,384,384,4.0,4.52,FFS,T2


In [37]:
# files_df.rename(columns={'patient': 'study_id', 'series': 'series_id', 'image': 'instance'}, inplace=True)

In [38]:
files_df['ss_id'] = files_df.apply(lambda row: f'{str(row["study_id"])}_{str(row["series_id"])}', axis=1)
files_df['instance_id'] = files_df.apply(lambda row: f'{str(row["study_id"])}_{str(row["series_id"])}_{str(row["instancenumber"])}', axis=1)

In [39]:
source_dir = CFG.PNG_DIR
files_df['filename'] = files_df.apply(lambda row: f'{source_dir}\\{row.study_id}_{row.series_id}_{row.image}.png', axis=1)

In [40]:
files_df.sample()

,study_id,series_id,image,instancenumber,rows,columns,slicethickness,spacingbetweenslices,patientposition,seriesdescription,ss_id,instance_id,filename
85085,3237383375,3153859504,8,8,512,512,4.0,5.0,HFS,T2,3237383375_3153859504,3237383375_3153859504_8,E:\data\RSNA2024\original\pngs_256\3237383375_...


In [41]:
train_desc_df.sample()

,study_id,series_id,series_description,ss_id
5050,3451042269,397282761,Sagittal T1,3451042269_397282761


In [42]:
coords_df = pd.merge(coords_df, files_df[['instance_id', 'rows', 'columns', 'filename']], left_on='instance_id', right_on='instance_id')

In [43]:
coords_df.sample()

,study_id,series_id,instance,condition,level,x,y,ss_id,instance_id,cl,plane,rows,columns,filename
26724,2397650165,1258166411,20,LSS,L5S1,282.732743,270.725664,2397650165_1258166411,2397650165_1258166411_20,LSSL5S1,Axial T2,512,512,E:\data\RSNA2024\original\pngs_256\2397650165_...


In [43]:
# TODO: make sure we match coords corectly
coords_df['x_perc'] = coords_df['x'] / coords_df['columns']
coords_df['y_perc'] = coords_df['y'] / coords_df['rows']

In [44]:
ax, non_ax = coords_df[coords_df['plane'] == 'Axial T2'], coords_df[coords_df['plane'] != 'Axial T2']
ax.shape, non_ax.shape, coords_df.shape

((19220, 16), (29472, 16), (48692, 16))

In [45]:
coords_df.y.min()

2.063097514340344

In [46]:
for c in [ax, non_ax]:
    print(c['x_perc'].min(), c['y_perc'].min())
    print(c['x_perc'].max(), c['y_perc'].max())
    print('/////////////')
    # print(c['x_perc'].mean(), c['y_perc'].mean())

0.2849557522123894 0.3092621664050236
0.6631964653474212 0.8078096961632665
/////////////
0.0048828125 0.0040294873326959845
0.7684669901065448 0.9117647058823529
/////////////


In [47]:
coords_df.shape

(48692, 16)

In [48]:
files_df.sample(5)

,study_id,series_id,instance,rows,columns,ss_id,instance_id,filename
102774,3703663905,1617588779,2,320,320,3703663905_1617588779,3703663905_1617588779_2,E:\data\RSNA2024\original\pngs_256\3703663905_...
15964,1425509806,57120628,34,640,640,1425509806_57120628,1425509806_57120628_34,E:\data\RSNA2024\original\pngs_256\1425509806_...
95601,3489581041,3869083782,15,320,320,3489581041_3869083782,3489581041_3869083782_15,E:\data\RSNA2024\original\pngs_256\3489581041_...
36201,1972541574,1065990522,30,640,640,1972541574_1065990522,1972541574_1065990522_30,E:\data\RSNA2024\original\pngs_256\1972541574_...
13026,1336127283,1679037242,15,320,320,1336127283_1679037242,1336127283_1679037242_15,E:\data\RSNA2024\original\pngs_256\1336127283_...


### Save results

In [98]:
train_df.to_csv(CFG.DEST_FOLDER / 'train.csv', index=False)

In [51]:
coords_df.to_csv(CFG.DEST_FOLDER / 'train_label_coordinates.csv', index=False)

In [133]:
files_df.to_csv(CFG.DEST_FOLDER / 'train_files.csv', index=False)